In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [46]:
class Residual(nn.Module): 
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        # conv2's shape must be as same as conv1. If not, then it will affect shape of kernel size. Because conv3's size is the same as conv1. So we can't change conv2's strides
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1,)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        print("x.shape",X.shape)
        Y = F.relu(self.bn1(self.conv1(X)))
        print("y.shape",Y.shape)
        Y = self.bn2(self.conv2(Y))
        print("y2.shape",Y.shape)
        if self.conv3:
            X = self.conv3(X)
        print("x2.shape",X.shape)
        Y += X
        return F.relu(Y)

# Test

In [49]:
X = torch.rand(4, 3, 6, 6)

# blk = Residual(3,3)
# Y = blk(X)
# print(Y.shape)

blk = Residual(3,6, use_1x1conv=True, strides=2)
blk(X).shape

print(blk)

x.shape torch.Size([4, 3, 6, 6])
y.shape torch.Size([4, 6, 3, 3])
y2.shape torch.Size([4, 6, 3, 3])
x2.shape torch.Size([4, 6, 3, 3])
Residual(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(3, 6, kernel_size=(1, 1), stride=(2, 2))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [50]:
from torchsummary import summary

In [52]:
blk = blk.to('cuda')
summary(blk, (3, 6, 6))

x.shape torch.Size([2, 3, 6, 6])
y.shape torch.Size([2, 6, 3, 3])
y2.shape torch.Size([2, 6, 3, 3])
x2.shape torch.Size([2, 6, 3, 3])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1              [-1, 6, 3, 3]             168
       BatchNorm2d-2              [-1, 6, 3, 3]              12
            Conv2d-3              [-1, 6, 3, 3]             330
       BatchNorm2d-4              [-1, 6, 3, 3]              12
            Conv2d-5              [-1, 6, 3, 3]              24
Total params: 546
Trainable params: 546
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [4]:
x = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

# 计算卷积核的边长和步幅
kernel_size = (3, 3)
stride = (1, 1)

# 创建卷积核
kernel = torch.tensor([[1, 0, -1], [2, 0, -2], [1, 0, -1]])

# 计算输入的形状
x.shape

torch.Size([3, 4])

# ResNet

In [5]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [55]:
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
  # b2's first residual donn't need stride=2 and maxpool because in b1 had already dont that. Others blocks' fisrt redisual would reduce the size of kernel.
  # In short, each block apply redisual twice, each block besides the first one would reduce the size once.
    blk = []
    for i in range(num_residuals):
        print(i)
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [56]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

0
1
0
1
0
1
0
1


In [8]:
net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 10))

In [9]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])
